In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from matplotlib import animation, colors
from sklearn.cluster import KMeans
from sklearn import preprocessing as sk_preprocessing
from sklearn.metrics import accuracy_score #scoring
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from tensorflow.keras import datasets, layers, models
from keras.callbacks import EarlyStopping

In [2]:
pressure = '/Users/votri/Downloads/DSDA 385 Machine Learning/Final Project - Climate Change/Pressure/MinMaxNormWithLabel/'

# Get the list of subfolders
subfolders = [subfolder for subfolder in os.listdir(pressure) if os.path.isdir(os.path.join(pressure, subfolder))]

dfs = []
reshaped_arrays = []
folder_names = []

for subfolder in subfolders:
    subfolder_path = os.path.join(pressure, subfolder)
    files = [f for f in os.listdir(subfolder_path) if f.endswith('.csv')]
    for file in files:
        csv_file_path = os.path.join(subfolder_path, file)
        df = pd.read_csv(csv_file_path, header=None)
        dfs.append(df)
        
        reshaped_array = np.reshape(df.values, (1, -1)) # Reshape the dataframe into a 1D array with 225 attributes
        reshaped_arrays.append(reshaped_array)  # Append the reshaped array to the list
        
        folder_names.append(subfolder) # Save the class of the image
        
# Concatenate the reshaped arrays into a single array
concatenated_array = np.concatenate(reshaped_arrays, axis=0)

# Convert the concatenated array into a dataframe
result_df = pd.DataFrame(concatenated_array)
result_df['Class'] = folder_names

In [3]:
print(result_df)
result_df.describe()

            0         1         2         3         4         5         6  \
0    0.139692  0.139077  0.137846  0.137231  0.137231  0.137231  0.137846   
1    0.475692  0.473846  0.471385  0.468923  0.465846  0.462769  0.460308   
2    0.248615  0.243692  0.240000  0.237538  0.235692  0.234462  0.234462   
3    0.248615  0.243692  0.240000  0.237538  0.235692  0.234462  0.234462   
4    0.293538  0.284923  0.278154  0.272615  0.268923  0.265231  0.262769   
..        ...       ...       ...       ...       ...       ...       ...   
732  0.844923  0.857231  0.872615  0.886154  0.890462  0.892308  0.890462   
733  0.912615  0.915692  0.923692  0.929846  0.932308  0.935385  0.936615   
734  0.931077  0.933538  0.931692  0.931692  0.931077  0.929231  0.928615   
735  0.868923  0.875077  0.880000  0.883692  0.884923  0.884923  0.883692   
736  0.856000  0.864000  0.874462  0.876923  0.873846  0.864000  0.868923   

            7         8         9  ...       216       217       218  \
0  

,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,224
count,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,...,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000
mean,0.653576,0.652917,0.652001,0.650932,0.649845,0.648673,0.647626,0.646720,0.646113,0.645782,...,0.649868,0.644940,0.641662,0.640246,0.640848,0.643169,0.647061,0.652169,0.658171,0.664773
std,0.259836,0.261524,0.262980,0.264169,0.265151,0.265783,0.266180,0.266394,0.266370,0.266135,...,0.277650,0.278376,0.278900,0.279298,0.279582,0.279557,0.279201,0.278348,0.277009,0.275190
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.483692,0.480000,0.478769,0.473846,0.468308,0.468308,0.467692,0.464615,0.464000,0.464615,...,0.459077,0.451692,0.448000,0.444308,0.444923,0.447385,0.451077,0.459692,0.467692,0.478154
50%,0.719385,0.721231,0.721231,0.716923,0.710769,0.711385,0.712615,0.705231,0.704000,0.701538,...,0.719385,0.710154,0.707077,0.707077,0.710769,0.709538,0.713231,0.719385,0.726154,0.736000
75%,0.882462,0.888615,0.894769,0.899692,0.901538,0.903385,0.902769,0.900923,0.898462,0.898462,...,0.910769,0.907077,0.902769,0.899692,0.900308,0.903385,0.910154,0.916308,0.918769,0.922462
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999385,0.995692,0.992615,0.988923,...,0.990769,0.994462,0.996923,0.996923,0.996923,0.995077,0.992000,0.988308,0.986462,0.987692


In [4]:
counts = result_df['Class'].value_counts()
print(counts)

COL     327
CL      178
COH     161
NROI     71
Name: Class, dtype: int64


In [5]:
X = result_df.drop('Class', axis = 1)
Y = result_df['Class']

Y_numerized = Y.replace({'CL': 0, 'COL' : 2, 'COH': 3, 'NROI': 1})
print(Y_numerized)

0      0
1      0
2      0
3      0
4      0
      ..
732    1
733    1
734    1
735    1
736    1
Name: Class, Length: 737, dtype: int64


In [6]:
X_2D = X.to_numpy().reshape(X.shape[0], 15, 15, 1)
print(X_2D)

[[[[0.139692]
   [0.139077]
   [0.137846]
   ...
   [0.150154]
   [0.153846]
   [0.156923]]

  [[0.136   ]
   [0.134154]
   [0.131692]
   ...
   [0.138462]
   [0.142154]
   [0.146462]]

  [[0.132308]
   [0.129846]
   [0.126154]
   ...
   [0.124923]
   [0.129846]
   [0.134769]]

  ...

  [[0.150154]
   [0.138462]
   [0.125538]
   ...
   [0.069538]
   [0.075692]
   [0.084308]]

  [[0.158769]
   [0.146462]
   [0.133538]
   ...
   [0.076923]
   [0.081846]
   [0.091692]]

  [[0.167385]
   [0.155692]
   [0.142769]
   ...
   [0.086154]
   [0.092308]
   [0.102769]]]


 [[[0.475692]
   [0.473846]
   [0.471385]
   ...
   [0.446769]
   [0.448   ]
   [0.450462]]

  [[0.473846]
   [0.470769]
   [0.467692]
   ...
   [0.437538]
   [0.44    ]
   [0.443077]]

  [[0.468923]
   [0.464615]
   [0.460308]
   ...
   [0.428308]
   [0.430769]
   [0.435692]]

  ...

  [[0.427077]
   [0.423385]
   [0.419077]
   ...
   [0.402462]
   [0.408615]
   [0.417846]]

  [[0.432615]
   [0.428923]
   [0.426462]
   ...
   [0

In [7]:
# train and test split
train_x, test_x, train_y, test_y = train_test_split(X_2D, Y_numerized, test_size=0.3, random_state=42)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape) # check the shapes

(515, 15, 15, 1) (515,) (222, 15, 15, 1) (222,)


In [8]:
# define the model architecture
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(15, 15)))
model.add(Dense(units=4, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
early_stopping = EarlyStopping(monitor='loss', patience=50)
model.fit(train_x, train_y, epochs=1000, batch_size=32, callbacks=[early_stopping])

Epoch 1/1000
17/17 [==============================] - 1s 4ms/step - loss: 1.1924 - accuracy: 0.5301
Epoch 2/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.9961 - accuracy: 0.5806
Epoch 3/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.9113 - accuracy: 0.5728
Epoch 4/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.8899 - accuracy: 0.6194
Epoch 5/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.8809 - accuracy: 0.6117
Epoch 6/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.8521 - accuracy: 0.6330
Epoch 7/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.8612 - accuracy: 0.6175
Epoch 8/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.8575 - accuracy: 0.6097
Epoch 9/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.8666 - accuracy: 0.6078
Epoch 10/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.8298 - accuracy: 0.6369

17/17 [==============================] - 0s 4ms/step - loss: 0.4125 - accuracy: 0.8350
Epoch 83/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3838 - accuracy: 0.8291
Epoch 84/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3554 - accuracy: 0.8466
Epoch 85/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4208 - accuracy: 0.7942
Epoch 86/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4182 - accuracy: 0.8155
Epoch 87/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3788 - accuracy: 0.8369
Epoch 88/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.6776 - accuracy: 0.7456
Epoch 89/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.4896 - accuracy: 0.8136
Epoch 90/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4057 - accuracy: 0.8291
Epoch 91/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.5492 - accuracy: 0.7515
Epoc

17/17 [==============================] - 0s 4ms/step - loss: 0.3375 - accuracy: 0.8427
Epoch 163/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4024 - accuracy: 0.8311
Epoch 164/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3881 - accuracy: 0.8233
Epoch 165/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3401 - accuracy: 0.8544
Epoch 166/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.4831 - accuracy: 0.7767
Epoch 167/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3838 - accuracy: 0.8330
Epoch 168/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3356 - accuracy: 0.8524
Epoch 169/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3481 - accuracy: 0.8388
Epoch 170/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3754 - accuracy: 0.8155
Epoch 171/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3422 - accuracy: 0.

17/17 [==============================] - 0s 4ms/step - loss: 0.3405 - accuracy: 0.8388
Epoch 243/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3034 - accuracy: 0.8757
Epoch 244/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3108 - accuracy: 0.8621
Epoch 245/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3145 - accuracy: 0.8408
Epoch 246/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3158 - accuracy: 0.8544
Epoch 247/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3150 - accuracy: 0.8524
Epoch 248/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.2933 - accuracy: 0.8718
Epoch 249/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3137 - accuracy: 0.8388
Epoch 250/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3186 - accuracy: 0.8738
Epoch 251/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3004 - accuracy: 0.

17/17 [==============================] - 0s 4ms/step - loss: 0.3027 - accuracy: 0.8680
Epoch 323/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3468 - accuracy: 0.8718
Epoch 324/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3345 - accuracy: 0.8485
Epoch 325/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3102 - accuracy: 0.8738
Epoch 326/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.3589 - accuracy: 0.8427
Epoch 327/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3256 - accuracy: 0.8388
Epoch 328/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.2979 - accuracy: 0.8602
Epoch 329/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3371 - accuracy: 0.8505
Epoch 330/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3516 - accuracy: 0.8427
Epoch 331/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.3244 - accuracy: 0.

In [10]:
# Use the trained model to make predictions on the test set
y_pred = model.predict(test_x).argmax(axis=1)

# Calculate the accuracy of the model on the test set
accuracy = accuracy_score(test_y, y_pred)
print('Test accuracy:', accuracy)

Test accuracy: 0.8783783783783784


In [11]:
cm = confusion_matrix(test_y, y_pred)
print(cm)

[[52  0  6  0]
 [ 0 17  1  3]
 [ 8  7 77  0]
 [ 0  2  0 49]]
